#MODEL IMPROVEMENT

In [ ]:
#Package that need to be installed
!pip install open3d

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.5/422.5 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 74.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 KB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.0 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
  Attempting uninstall: nbformat
    Found existing installation: nbformat 5.7.1
    Uninstalling nbformat-5.7.1:
      Successfully uninstalled nbformat-5.7.1


In [ ]:
import numpy as np
import pandas as pd
import open3d
import os
import torch
import torch.nn as nn

from torch.utils.data import DataLoader
from keras.utils import to_categorical
import tensorflow as tf
from sklearn import metrics
from sklearn.model_selection import train_test_split

from keras.models import Sequential, Model
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import Input
from keras.layers import Conv3D
from keras.layers import MaxPool3D
from keras.layers import Flatten
from keras.layers import Concatenate
from keras.callbacks import EarlyStopping

# DATA PREPARATION
Take the voxel Grid file and trasform it into 3D numpy array of dimensions (32,32,32,1) in order to use them as input for the CNN


In [ ]:
#Prepare DATA
class_folders = os.listdir('/content/drive/MyDrive/ModelNet10_Voxel')

print(class_folders)
i = -1


Labels:
night_stand 0
bathtub 1
monitor 2
dresser 3
table 4
bed 5
chair 6
sofa 7
toilet 8
desk 9


In [ ]:
def cutout_3d(volume, cutout_size=(8,8,8)):
    # Shape is 32x32x32
    x = np.random.randint(0, 32 - cutout_size[0])
    y = np.random.randint(0, 32 - cutout_size[1])
    z = np.random.randint(0, 32 - cutout_size[2])
    cutout_cube = (x, y, z, cutout_size[0], cutout_size[1], cutout_size[2])
    volume[x:x+cutout_size[0], y:y+cutout_size[1], z:z+cutout_size[2]] = 0
    return volume

In [ ]:
#Extrapolate the voxels from the file 
def process_off_file(filepath):

    voxel = open3d.io.read_voxel_grid(filepath)
    return voxel

#Prepare DATA
train_data_raw = []
train_labels = []
test_data_raw = []
test_labels = []

i = -1
#Extrapolate data from the dataset
for class_folder in class_folders:
  train_files = os.listdir(f'/content/drive/MyDrive/ModelNet10_Voxel/{class_folder}/train')
  test_files = os.listdir(f'/content/drive/MyDrive/ModelNet10_Voxel/{class_folder}/test')
  i = i + 1
  for train_file in train_files:
      
      filepath = f'/content/drive/MyDrive/ModelNet10_Voxel/{class_folder}/train/{train_file}'
      points = process_off_file(filepath)
      train_data_raw.append(points)
      train_labels.append(i)
      

  for test_file in test_files:
      filepath = f'/content/drive/MyDrive/ModelNet10_Voxel/{class_folder}/test/{test_file}'
      points = process_off_file(filepath)
      test_data_raw.append(points)
      test_labels.append(i)

['night_stand', 'bathtub', 'monitor', 'dresser', 'table', 'bed', 'chair', 'sofa', 'toilet', 'desk']


In [ ]:
# Transform the Voxel Grid into Numpy Array containing a list of voxels 
train_data_numpy = []
test_data_numpy = []

#First step: Each array to voxels 
for i in range(len(train_data_raw)):
  train_data_numpy.append(np.asarray(train_data_raw[i].get_voxels()))

for i in range(len(test_data_raw)):
  test_data_numpy.append(np.asarray(test_data_raw[i].get_voxels()))

test_data_raw = []
train_data_raw = []
#print(test_data_numpy[0])

In [ ]:
#For each sample create an Array of dimension (32,32,32,1) with true where voxels are
array_train_ready = []
array_test_ready = []

for i in range(len(train_data_numpy)):
  array_temp = np.zeros((32, 32, 32,1))
  for j in range(len(train_data_numpy[i])):
      array_temp[train_data_numpy[i][j].grid_index[0],train_data_numpy[i][j].grid_index[1],train_data_numpy[i][j].grid_index[2]] = 1
  array_train_ready.append(array_temp)

for i in range(len(test_data_numpy)):
  array_temp = np.zeros((32, 32, 32,1))
  for j in range(len(test_data_numpy[i])):
      array_temp[test_data_numpy[i][j].grid_index[0],test_data_numpy[i][j].grid_index[1],test_data_numpy[i][j].grid_index[2]] = 1
  array_test_ready.append(array_temp)

In [ ]:
array_train_ready = np.asarray(array_train_ready) #len 3950
print(len(array_train_ready))
array_test_ready = np.asarray(array_test_ready) #len 908
print(len(array_test_ready))

3621
908


# RETE PIU' PICCOLA DI TEST


In [ ]:
from keras.layers import Conv3D, MaxPooling3D, Flatten, Dense
from keras.models import Sequential

# create model
model = Sequential()

# add convolutional layers
model.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(32, 32, 32, 1), activation='relu'))
model.add(MaxPooling3D(pool_size=(3, 3, 3)))
model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(3, 3, 3)))

# add flatten layer
model.add(Flatten())

# add dense layers
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
#Add a shuffle for array array_ready !!
#New train with callback (implemented)
#Add a little test set using a couple example for each 

num_classes = 10
# load your dataset of voxel grids and labels
X_train = np.array(array_train_ready)
y_train = np.array(train_labels)
X_test = np.array(array_test_ready)
y_test = np.array(test_labels)

#Shuffle the train set
idx = np.random.permutation(len(X_train))
X_train, y_train = X_train[idx], y_train[idx]

#Shuffle the validation set
idt = np.random.permutation(len(X_test))
X_test, y_test = X_test[idt], y_test[idt]

#Train set contains 80% of data and test set 20%
#We will use half of the test set for validation, and half for the test
#Approximately 80/10/10

half_set = int((len(X_test)/2))

X_validation = X_test[:half_set]
y_validation = y_test[:half_set]

X_test = X_test[half_set:]
y_test = y_test[half_set:]

# convert labels to categorical format
y_train = to_categorical(y_train, num_classes)
y_validation = to_categorical(y_validation, num_classes) 
y_test = to_categorical(y_test, num_classes)

print("Train test: " + str(len(y_train)))
print("Validation test: " + str(len(y_validation)))
print("Test test: " + str(len(y_test)))

Train test: 3621
Validation test: 454
Test test: 454


In [ ]:
#From the 4000 samples create a random 1000 CUTOUT 
#FROM train samples 15% CUTOUT
#Cutout is a data augmentation technique that randomly masks out rectangular regions of input arrays. 
#This can help the model to learn to ignore irrelevant features and generalize better.

for i in range(int(len(X_train)*0.25)):
  X_train[i] = cutout_3d(X_train[i])

idx = np.random.permutation(len(X_train))
X_train, y_train = X_train[idx], y_train[idx]

#print(y_train[:10])

In [ ]:
from keras.callbacks import ModelCheckpoint

# Create a ModelCheckpoint object that saves the model's weights only when the accuracy improves
checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_accuracy', mode='max', verbose=1)


In [ ]:
# Train the model and pass the ModelCheckpoint object as a callback
history = model.fit(X_train, y_train, epochs=15, validation_data=(X_validation, y_validation), callbacks=[checkpoint])

Epoch 1/15
114/114 [==============================] - ETA: 0s - loss: 0.8808 - accuracy: 0.7189
Epoch 1: val_accuracy improved from -inf to 0.78855, saving model to best_model.h5
114/114 [==============================] - 12s 30ms/step - loss: 0.8808 - accuracy: 0.7189 - val_loss: 0.5572 - val_accuracy: 0.7885
Epoch 2/15
112/114 [============================>.] - ETA: 0s - loss: 0.3695 - accuracy: 0.8795
Epoch 2: val_accuracy improved from 0.78855 to 0.83921, saving model to best_model.h5
114/114 [==============================] - 3s 26ms/step - loss: 0.3702 - accuracy: 0.8793 - val_loss: 0.4468 - val_accuracy: 0.8392
Epoch 3/15
113/114 [============================>.] - ETA: 0s - loss: 0.2513 - accuracy: 0.9145
Epoch 3: val_accuracy improved from 0.83921 to 0.87004, saving model to best_model.h5
114/114 [==============================] - 3s 24ms/step - loss: 0.2517 - accuracy: 0.9144 - val_loss: 0.4427 - val_accuracy: 0.8700
Epoch 4/15
113/114 [============================>.] - ETA: 0

In [ ]:
from keras.models import load_model
#Save model
model.save('model.h5')
#Save weights
model.save_weights('model_weights.h5')

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print("Loss on test set: " + str(score[0]))
print("Accuracy on test set: " + str(score[1]))

Loss on test set: 0.4686940610408783
Accuracy on test set: 0.8810572624206543


In [ ]:
tensor = []
tensor.append(X_test[0])
print(y_test[0])
tensor = np.asarray(tensor)
print(tensor.shape)
y = model.predict(tensor)
for i in range(len(y)):
  print(y[i])

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
(1, 32, 32, 32, 1)
1/1 [==============================] - 0s 14ms/step
[2.8176753e-10 1.6855187e-08 2.8985261e-09 4.4157681e-11 2.5736397e-09
 9.9999988e-01 9.2570656e-12 1.4413493e-07 4.6326733e-08 9.7084811e-09]
